In [1]:
import tensorflow as tf

C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (

In [2]:
# 플로어를 작성한 것
hi = tf.constant('hello')
hi

<tf.Tensor 'Const:0' shape=() dtype=string>

In [3]:
# 파이썬 바깥에서 구동한 것
sess = tf.Session()
sess.run(hi)

b'hello'

#### 텐서플로우를 이용한 간단한 계산

- 덧셈



In [4]:
# 동일 코드(프로그램)안에서 연속적으로 구동하면 count가 된다(가중) -> 그래프 같은 것들이 계속 구동할 때마다 바뀐다
# 상수 정의
a = tf.constant(1234)
b = tf.constant(5000)
a, b

(<tf.Tensor 'Const_1:0' shape=() dtype=int32>,
 <tf.Tensor 'Const_2:0' shape=() dtype=int32>)

In [5]:
# 두번정도 구동하면 add_0:0 -> add_1:0 으로 바뀜
# 계산 정의
add_op = a + b
add_op

<tf.Tensor 'add:0' shape=() dtype=int32>

- 텐서플로우는 덧셈이라는 계산을 정의만 할 뿐, 변수에 add_op 에 대입되는 것은
- 덧셈의 결과로 나오는 것은 상수가 아니라 데이터 플로우 그래프라는 객체이다. (전체적맥락은 덧셈을 한다고 숫자가 들어오는 것은 아니다)

In [6]:
# 세션을 시작
sess = tf.Session()

In [7]:
# 계산처리는 파이썬이 하지 않고, 파이썬 외부 즉, 텐서플로우에서 진행
# 계산처리는 그래프라는 객체를 구축하고, 이러한 그래프를 텐서플로우가 실행하면서 데이터가 흘러가는 구조이다
# (run을 해야 돌아간다) -> 그전까지는 틀을 작성한다는 느낌으로 생각하면 된다
res = sess.run(add_op)
res, print(type(res))

<class 'numpy.int32'>


(6234, None)

#### 텐서플로우의 기본 항목

- 핵심 데이터 구조 요소 3개
 > 상수          : Constant
 > 변수          : Variable
 > 플레이스 홀더 : Placeholder
 


In [8]:
# a_0:0 -> a_1:0 (두번가동시)
# 상수 정의
a = tf.constant(120, name='a')
b = tf.constant(130, name='b')
c = tf.constant(140, name='c')

a, a.name, b, b.name, c, c.name

(<tf.Tensor 'a:0' shape=() dtype=int32>,
 'a:0',
 <tf.Tensor 'b:0' shape=() dtype=int32>,
 'b:0',
 <tf.Tensor 'c:0' shape=() dtype=int32>,
 'c:0')

In [9]:
# 변수 정의
v = tf.Variable(0, name='v')
v, v.name

Instructions for updating:
Colocations handled automatically by placer.


(<tf.Variable 'v:0' shape=() dtype=int32_ref>, 'v:0')

In [10]:
# 데이터 플로우 그래프 정의
calc_op = a + b + c
calc_op

<tf.Tensor 'add_2:0' shape=() dtype=int32>

In [11]:
# 변수에 값을 대입 => 데이터 플로우 그래프
assign_op = tf.assign(v, calc_op)
assign_op

<tf.Tensor 'Assign:0' shape=() dtype=int32_ref>

In [12]:
# 세션 실행
sess = tf.Session()
sess.run(assign_op) # calc_op가 물려있어서 같이 진행될 것이다

390

In [13]:
v # 얘는 그냥 변수이다. 값을 확인할 방법이 없다.-> 밖으로 나가서 가동해야 한다

<tf.Variable 'v:0' shape=() dtype=int32_ref>

In [14]:
# 변수값 출력하기 -> 세션 수행을 통해서 확인 (run 전까지는 플로우를 짜라)
v, sess.run(v)

(<tf.Variable 'v:0' shape=() dtype=int32_ref>, 390)

- 플레이스홀더
 > 쿼리 수행할 때, ? 기호 <= 이런 느낌
 > 데이터 플로우 그래프 구축시, 값을 넣지 않고, 값을 담을 수 있는 형태의 그릇만 만드는 형식 => 세션이 수행될 때 값을 넣어서 실행
 > 템플릿과 유사하게 값을 넣어 두는 공간을 만드는 기능


In [15]:
# 정수형 데이터를 3개를 담을 수 있는 그릇(배열이용) 준비 -> 플레이스 홀더를 준비
a = tf.placeholder(tf.int32, [3])

In [16]:
# 상수
b = tf.constant(2)

In [17]:
# 데이터 플로우 그래프
# 방 3개짜리에 상수2개를 곱해줌
# 벡터에다가 스칼라를 곱하는 꼴이다
# 배열 * 값 => 행렬 * 값 => 백터 * 스칼라

x2_op = a * b 

In [18]:
sess = tf.Session()

In [19]:
# 수행 
# b는 상수값이 들어있지만, a는 방만 있고 안이 비어있는 상태이다 -> feed_dict= : 에 값을 채운다
# feed_dict= 는 플레이스 홀데에 값을 채운다
r1 = sess.run(x2_op, feed_dict={a:[1,2,3]} ) 
r1 # 결과값은 b의 상수 2와 a의 값을 곱하니깐 2,4,6이 출력된다

array([2, 4, 6])

#### 배열의 크기를 가변으로 처리하여 구성



In [20]:
# None을 넣으면 가변 처리가 되서, 배열의 개수는 가변적으로 대입 가능
# 플레이스 홀더 정의
a = tf.placeholder(tf.int32, [None])  # 변수의 개수가 몇개 올지 모르겠을 때, None(가변)을 넣는다

In [21]:
# 상수 정의
b = tf.constant(10)

In [22]:
# 데이터 플로우 그래프 정의 
x10_op = a * b

In [23]:
sess = tf.Session()

In [24]:
# 플레이스 홀더가 가변 처리되서 a의 값이 여러개가 와도 연산이 가능하다
sess.run(x10_op, feed_dict={a:[1,2,3,4,5]})

array([10, 20, 30, 40, 50])

In [25]:
# 플레이스 홀더가 가변 처리되서 a의 값이 여러개가 와도 연산이 가능하다
sess.run(x10_op, feed_dict={a:[10,20]})

array([100, 200])